# Chp 9. Two Dimensions

- 발표자 : 이아름
- 발표일 : 2017.9.16(토)

## 9.1 Paintball

- 페인트볼 게임 : 서로에게 총을 쏘는 게임
    
    - 30피트 * 50피트 인 실내 경기장에서 게임을 할 때, 당신이 30피트 벽 옆에 서 있고 근처에 숨어있는 사람을 찾고있다.

    - 벽에 같은 색의 페인트 자국이 있는데 실내 경기장 왼쪽 아래에서 부터 15,16,18,21피트 거리에 있다. 
    지금까지 데이터로 추정했을 때, 상대편이 어디쯤 숨어있는 것 같은가?




<img src="./figures/01.png" width=300 />




- 위의 그림은 경기장의 다이어그램이다. 

    - 왼쪽 아래가 원점. 
    - a,b 좌표에 총을 쏜 사람이 있다고 나타낸다.
    - 페인트 자국의 위치는 x로 표시
    - 상대편이 발사한 각도는 θ
    
    
## 9.2 The suite

- 상대방 위치에 대한 가설 집합을 나타내는 suite 만들기
    - 가설은 좌표 (a,b) 순서쌍
    - 페인트볼의 수윗 정의는 다음과 같다

In [1]:
from __future__ import print_function, division

% matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import math
import numpy as np

from thinkbayes2 import Pmf, Cdf, Suite, Joint
import thinkplot
import thinkbayes2

In [2]:
class Paintball(thinkbayes2.Suite, thinkbayes2.Joint):
    def __init__(self, alphas, betas, locations):
        self.locations = locations
        pairs = [(alpha, beta)
            for alpha in alphas
            for beta in betas]
        thinkbayes2.Suite.__init__(self, pairs)

    - paintball은 suite과 joint 상속받는다
    - alphas betas : 각 알파 베타 값의 리스트
    - locations 은 벽을 따라 가능한 모든 장소의 리스트 <- likelihood 가 사용함
    
- suite 만들기 코드는 아래와 같다

In [3]:
alphas=range(0,31)
betas = range(1, 51)
locations = range(0, 31)
suite = Paintball(alphas, betas, locations)

## 9.3 Trigonometry

- 주어진 상대방의 위치에 대해, 벽에 어떻게 자국을 남길 수 있을지에 대한 우도를 계산해야함 -> 우도 함수가 필요함

- 상대방이  θ의 각도로 총을 쏘았을 때, 총알은 x위치 벽을 맞출 것이고 식은 아래와 같다.


<img src="./figures/02.png" width=200 />

    - 벽의 위치가 주어지면 θ 구할 수 있다.
    - θ에 대해 미분
    
<img src="./figures/03.png" width=200 />


    - θ가 증가함에 따라 대상 위치에 대한 속도에 대한 것으로 strafing speed(총격속도) 라고 함
    - 벽의 주어진 점을 맞출 확률은 총격 속도의 역수에 비례함!
    
    - 총을 쏜 사람, 벽의 위치좌표 알면은 strafing speed 계산 가능
    
    - alpha와 beta는 총을 쏜 사람의 좌표
    - x는 벽에 난 자국의 위치
    - 결과는 theta에 대한 x의 미분값!

In [4]:
def StrafingSpeed(alpha, beta, x):
    theta = math.atan2(x - alpha, beta)
    speed = beta / math.cos(theta)**2
    return speed

    - 벽의 어떤 위치를 맞출 확률을 나타내는 pmf 계산
    - 이때, 위에서 주어진 locations 사용

In [5]:
def MakeLocationPmf(alpha, beta, locations):
    pmf = thinkbayes.Pmf()
    for x in locations:
        prob = 1.0 / StrafingSpeed(alpha, beta, x)
        pmf.Set(x, prob)
    pmf.Normalize()
    return pmf

    - makelocationpmf는 총격 속도에 역으로 비례하는, 각 위치를 맞출 확률을 계산함.
    - 결과는 위치에 대한 pmf와 이에 해당하는 확률 임!
    
    - 아래 그림은, alpha =10 에 대해 beta 값별 위치 pmf 보여줌.
    - beta의 모든 값에 대해 페인트볼에 맞을 확률이 가장 높은 위치는 x=10 이다.
    - beta가 증가할수록 pmf 분포도 증가한다.
    
<img src="./figures/04.png" width=200 />


## 9.4 likelihood

- 우도 함수 계산하기

- makelocationpmf 사용해서 '상대방 좌표 x값'에 대한 우도 계산하기
    - alpha와 beta는 저격수에 대한 가설의 좌표. 
    - x는 관측된 페인트 자국의 위치
    
    -pmf는 주어진 저격수의 좌표에 대한 각 위치에 대한 확률이 들어있다.
        - 이 pmf에서 관측장소에 대한 확률 확인할 수 있음.

In [6]:
def Likelihood(self, data, hypo):
    alpha, beta = hypo
    x = data
    pmf = MakeLocationPmf(alpha, beta, self.locations)
    like = pmf.Prob(x)
    return like

    - update 사용해서 suite 갱신
    - 결과는 각 (a,b)쌍의 사후 확률 분포라고 할 수 있다

In [7]:
suite.UpdateSet([15,16,18,21])

UnimplementedMethodException: 

## 9.5 Joint distributions

- 분포의 각 값이 튜플로 되어 있을 때, 이는 변수가 함께 있는 상태의 분포를 나타내므로 joint distribution 이라고 하낟.
- joint distribution 에는 변수들의 분포와 변수 간의 관계에 대한 정보가 들어있음

- 결합 분포가 주어지면, 이를 사용해서 각 변수의 분포를 독립적으로 계산 -> 주변분포


In [8]:
# class Joint:
def Marginal(self, i):
    
    pmf = Pmf()
    for vs, prob in self.Items():
        pmf.Incr(vs[i], prob)
    return pmf

- i는 구하고자 하는 변수
- i=0은 알파 분포, i=1은 베타 분포

- 주변분포 계산하기

In [9]:
marginal_alpha = suite.Marginal(0)
marginal_beta = suite.Marginal(1)

- 변환된 결과를 보면, alpha 중간 값은 18, -> 관측된 자국의 위치 [15,16,18,21] 의 중간 근처
- beta 값은 가장 높은 값은 벽 근처이지만, 10피트 이상까진 거의 정규분포에 가깝다


<img src="./figures/07.png" width=400 />


- 사후 주변 분포를 사용해서, 독립적으로 신뢰구간 구할 수 있다

    - 알파 신뢰구간은 7,21
    - 베타 신뢰구간은 13,38

In [10]:
print ('alpha CI', marginal_alpha.CredibleInterval(50))
print ('beta CI', marginal_beta.CredibleInterval(50))

alpha CI (7, 23)
beta CI (13, 38)


## 9.6 Conditional distributions

- 주변분포는 변수가 독립적으로 담고 있기 때문에 변수의 종속성에 대해서는 알 수 없다.

- 조건분포(conditional distributions)를 계산해서 종속성을 시각화 할 수 있다.
    - i는 변수 인덱스
    - j는 조건 변수 인덱스
    - val은 조건값


In [11]:
def Conditional(self, i, j, val):
    pmf = Pmf()
    for vs, prob in self.Items():
        if vs[j] != val: continue
        pmf.Incr(vs[i], prob)
    pmf.Normalize()
    return pmf

In [12]:
betas=[10,20,40]
for beta in betas:
    cond = suite.Conditional(0, 1, beta)

- 변수가 독립적인 경우, 조건 분포는 항상 동일하다.
- 분포가 모무 다를 경우, 변수가 상호의존적이라고 볼 수 있다.
    - beta =10 이라는걸 알고 있으면, alpha의 조건분포는 좁아짐
    - beta가 더 크면 alpha의 분포는 더 넓어짐
    
    
<img src="./figures/05.png" width=400 />

## 9.7 Credible intervals

- 사후결합확률을 시각화 하는 다른 방법으로는 신뢰구간 계산 하는 것임
    - 주어진 분포에 대해, 신뢰도가 동일한 구간이 많이 있다는 것
        - ex) 50% 신뢰구간을 찾는 경우, 각 확률을 합쳐 50%가 되는 어떤 값의 집합이든 선택할 수 있다.
    
    - 값이 1차원일 경우, 중심신뢰구간!을 선택하는 것이 일반적
        - ex) 중심부터 50%의 신뢰구간은 25번째부터 75번째까지의 구간의 모든 값을 포함함
        
    - 다차원의 경우, 신뢰구간이 50%가 될 수 있을 가능성이 가장 높은 값들을 포함하는 '최대 우도 신뢰구간'을 선택함.
    
    
    - maxlikinterval
        - 첫 단계는 확률이 큰 순서대로 suite 에 리스트로 넣기
        - 리스트로, 전체확률 percentage를 초과할 때까지 각 값을 구간에 더한다.
         
    

In [13]:
# class Joint:
def MaxLikeInterval(self, percentage=90):
    interval = []
    total = 0
    t = [(prob, val) for val, prob in self.Items()]
    t.sort(reverse=True)
    for prob, val in t:
        interval.append(val)
        total += prob
        if total >= percentage/100.0:
            break
    return interval

In [14]:
def MakeCrediblePlot(suite):
    d = dict((pair, 0) for pair in suite.Values())
    percentages = [75, 50, 25]
    for p in percentages:
        interval = suite.MaxLikeInterval(p)
        for pair in interval:
            d[pair] += 1
    return d

- makecredibleplot 를 사용하여 신뢰구간 시각화 할 수 있다.

- return d 는 suite의 각 리스트 값을 ,  이 값이 있는 구간에 연결해 주는 딕셔너리.
    - 반복문에서 각 비율에 대한 구간 계산하고 d 수행



<img src="./figures/06.png" width=400 />

- 위 이미지는 상대좌표에 대한 신뢰구간.
- 25% 신뢰구간은 벽 근처의 가장 어두운 부분, 
    - 비율이 높을수록 신뢰구간은 커지고, 경기장의 오른쪽으로 치우쳐 있다